## Scraper

Scrape webpages for DoB and DoD info.

Write compiled information to CSV

In [248]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
from datetime import datetime


## Make list of agent names

In [446]:
nameList=[### older
          # "Martin Luther",
          # "James Ussher",
          # "Gottfried Wilhelm Leibniz",
          # "René Descartes",
          # "Thomas Burnet", # bad format
          # "John Woodward (naturalist)", # bad format
          # "William Whiston",
        #  ### middle
        #   "Immanuel Kant",
        #   "Pierre-Simon Laplace",
        #   "Georges-Louis Leclerc, Comte de Buffon",
        #   "Robert Hooke",
        #   "Nicolas Steno",
        #   "Benoît de Maillet",
        #   "James Hutton",
        #   "Georges Cuvier",
        #   "Jean-Baptiste Lamarck",
        #   "Richard Kirwan",
        #   "Sir James Hall, 4th Baronet", # bad format
        #   "Abraham Gottlob Werner",
        #   "William Buckland",
        #   "George Julius Poulett Scrope",
        ### newer
          "Louis Agassiz",
          "William Whewell",
          "Archibald Geikie",
          "Thomas Chrowder Chamberlin",
          "Julius von Mayer",
          "James Prescott Joule",
          "Rudolf Clausius",
          "Lord Kelvin", 
          "Thomas Henry Huxley",
          "Samuel Wilberforce",
          "Charles Lyell",
          "Charles Darwin",
          "James Croll",
          "Albert Einstein"
        ]

## Scrape wikipedia

### Useful Functions

In [438]:
### get wikipedia info. box as text
def GetXmlInfo(inStr):

    nameStr="_".join(inStr.split(' '))
    url = f'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles={nameStr}&format=xml'

    # print(url)

    res = requests.get(url)
    soup = BeautifulSoup(res.text, "xml")

    try:
        return soup.query.pages.revisions.getText()
    except Exception as e:
        print("Retieve xml info went wrong")
        print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")
        return None

In [439]:
def PatternMatch(inStr):
    # print(inStr)
    patList=["%d/%m/%y","%d %B %Y","%B %d, %Y"]
    retStr=None
    for pat in patList:
        # print(pat)
        try:
            retStr=datetime.strptime(inStr, pat).strftime("%Y-%m-%d")
            break
        except ValueError:
            pass
    return retStr

### testing
PatternMatch("31 March 1596")

'1596-03-31'

In [472]:
def LastChance(infoStr, strList):

    for sl in strList:
        print(f"trying: {sl}")
        searchStr=re.search(r'('+sl+'(.*?))\n', infoStr) 
        print(searchStr)
        if searchStr!=None:
            date_data = searchStr.group(0).split('=')[1]
            date_data = date_data.split('(')[0].strip()#.split(' ')
            # print(date_data)
            dateStr=PatternMatch(date_data)
 
            if dateStr!=None:
                return ["gotit"]+dateStr.split('-')
            # print(dateStr)

# ### testing 
# retDate=LastChance(retStr, ["Birth date","birth date","birth_date"])
# print("this:",retDate)

In [473]:
### get birth and death dates from info. text
def GetLifeSpan(xmlInfo):
    birth_data, death_data= None, None

    ### birth date extraction
    # standard check
    for snip in [x+y+z for x in ['birth','Birth'] for y in [' ','-','_'] for z in ['date', 'Date']]:
        try:
            birth_re = re.search(r'('+snip+'(.*?)}})', xmlInfo)
            birth_data = birth_re.group(0).split('|')
            print(f"found {snip}: {birth_data}")
            break
        except AttributeError:
            pass
    # last chance check
    if birth_data==None:
        print("last chance try")
        birth_data=LastChance(xmlInfo, ["Birth date","birth date","birth_date"])
        print(f"- {birth_data}")
    # except Exception as e:
    #     print("Birth search went wrong")
    #     print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")
    
    if birth_data!=None:
        print(birth_data)
        birth_data=[bd.replace('{','').replace('}','') for bd in birth_data if "df" not in bd]
        if len(birth_data)<4:
            for bd in birth_data:
                bd_ed=bd.replace('{','').replace('}','')
                bd_mt=PatternMatch(bd_ed)
                if bd_mt!=None:
                    birth_data= ["gotit"]+bd_mt.split('-')
                    break
        # %Y-%m-%d
        print("Got Birth info (pre):",birth_data)
        birthStr=f"{birth_data[1]}-{birth_data[2]}-{birth_data[3]}"
        print("Got Birth info:",birthStr)
    else:
        birthStr=None

    ### death date extraction
    # standard check
    for snip in [x+y+z for x in ['death','Death'] for y in [' ','-','_'] for z in ['date', 'Date']]:
        try:
            death_re = re.search(r'('+snip+'(.*?)}})', xmlInfo)
            death_data = death_re.group(0).split('|')
            print(f"found {snip}: {death_data}")
            break
        except AttributeError:
            pass
    # last chance check
    if death_data==None:
        print("last chance try")
        death_data=LastChance(xmlInfo, ["Death date","death date","death_date"])
        print(f"- {death_data}")
    # except Exception as e:
    #     print("Death search went wrong")
    #     print(f"An exception of type {type(e).__name__} occurred. Details: {str(e)}")

    if death_data!=None:
        # print(death_data)
        death_data=[dd.replace('{','').replace('}','') for dd in death_data if "df" not in dd and "and" not in dd]
        if len(death_data)<4:
            for dd in death_data:
                dd_ed=dd.replace('{','').replace('}','')
                dd_mt=PatternMatch(dd_ed)
                if dd_mt!=None:
                    death_data= ["gotit"]+dd_mt.split('-')
                    break
        # %Y-%m-%d
        deathStr=f"{death_data[1]}-{death_data[2]}-{death_data[3]}"
        print("Got Death info:",deathStr)
    else:
        deathStr=None

    return birthStr, deathStr


### do Scraping

In [474]:

data=[]
for name in nameList:
    print(f"### {name}")
    
    data.append({'name':name})

    xmlInfo=GetXmlInfo(name)
    # print(xmlInfo)

    if xmlInfo==None:
        print("No xmlInfo returned. skipping")
        continue

    # print(xmlInfo)
    dob, dod = GetLifeSpan(xmlInfo)
    
    data[-1]['born']=dob
    data[-1]['died']=dod

    

### Louis Agassiz
found birth date: ['birth date', '1807', '05', '28', 'mf=y}}']
['birth date', '1807', '05', '28', 'mf=y}}']
Got Birth info (pre): ['birth date', '1807', '05', '28', 'mf=y']
Got Birth info: 1807-05-28
found death date: ['death date and age', '1873', '12', '14', '1807', '05', '28', 'mf=y}}']
Got Death info: 12-14-1807
### William Whewell
found birth date: ['birth date', '1794', '05', '24', 'df=y}}']
['birth date', '1794', '05', '24', 'df=y}}']
Got Birth info (pre): ['birth date', '1794', '05', '24']
Got Birth info: 1794-05-24
found death date: ['death date and age', '1866', '03', '06', '1794', '05', '24', 'df=y}}']
Got Death info: 03-06-1794
### Archibald Geikie
found birth_date: ['birth_date        = {{Birth date', '1835', '12', '28', 'df=yes}}']
['birth_date        = {{Birth date', '1835', '12', '28', 'df=yes}}']
Got Birth info (pre): ['birth_date        = Birth date', '1835', '12', '28']
Got Birth info: 1835-12-28
found death_date: ['death_date        = {{nowrap', '{

## Visualisation

### Table of data

In [475]:
df_data=pd.DataFrame(data)
df_data

,name,born,died
0,Louis Agassiz,1807-05-28,12-14-1807
1,William Whewell,1794-05-24,03-06-1794
2,Archibald Geikie,1835-12-28,1924-11-10
3,Thomas Chrowder Chamberlin,1843-09-25,1928-11-15
4,Julius von Mayer,1814-11-25,3-20-1814
5,James Prescott Joule,1818-12-24,10-11-1818
6,Rudolf Clausius,1822-1-02,08-24-1822
7,Lord Kelvin,1824-06-26,12-17-1824
8,Thomas Henry Huxley,1825-5-4,6-29-1825
9,Samuel Wilberforce,1805-09-07,1873-07-19


### Timeline chart

In [471]:
alt.Chart(df_data).mark_bar().encode(
    x='born:T',
    x2='died:T',
    y='name'
)

alt.Chart(...)

### test wiki info.

In [464]:
# name="Albert Einstein"
name="Julius von Mayer"
url = f"http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles={'_'.join(name.split(' '))}&format=xml"

res = requests.get(url)
soup = BeautifulSoup(res.text, "xml")

retStr=soup.query.pages.revisions.getText()
print(retStr)




{{Short description|German physician, chemist, and physicist}}
{{Use dmy dates|date=January 2020}}
{{distinguish|Julius Lothar Meyer}}
{{Infobox scientist
| name = Julius Robert von Mayer
| image = Julius Robert Mayer von Friedrich Berrer (portrait).jpg
| birth_name = Julius Robert Mayer<ref>Ennobled in 1867.</ref>
| birth_date = 25 November 1814
| birth_place = [[Heilbronn]]
| death_date = {{Death date and age|1878|3|20|1814|11|25|df=y}}
| death_place = Heilbronn, [[German Empire]]
| residencllllle = 
| citizenship = 
| nationality = German
| ethnicity = 
| field = [[Physics]]
| work_institutions = 
| alma_mater = [[University of Tübingen]]
| doctoral_advisor = 
| doctoral_students = 
| known_for = [[First law of thermodynamics]]<br>[[Mechanical equivalent of heat]]<br>[[Mayer's reagent]]<br>[[Mayer's relation]]
| influences = 
| influenced = [[William Thomson, 1st Baron Kelvin|William Thomson]]
| prizes = [[Copley Medal]] (1871)<br>[[Poncelet Prize|Poncelet Prize (1869)]]
}}

'''Juli

In [465]:
def LastChance(infoStr, strList):

    patList=["%d/%m/%y","%d %B %Y","%B %d, %Y"]

    for sl in strList:
        print(f"trying: {sl}")
        searchStr=re.search(r'('+sl+'(.*?))\n', infoStr) 
        print(searchStr)
        if searchStr!=None:
            date_data = searchStr.group(0).split('=')[1]
            date_data = date_data.split('(')[0].strip()#.split(' ')
            print(date_data)
            dateStr=None
            for pat in patList:
                print(pat)
                try:
                    dateStr=datetime.strptime(date_data, pat).strftime("%Y-%m-%d")
                    break
                except ValueError:
                    pass
            if dateStr!=None:
                return dateStr
            # print(dateStr)

# retDate=LastChance(retStr, ["Birth date","birth date","birth_date"])
retDate=LastChance(retStr, ["Death date","death date","death_date"])
print("this:",retDate)

trying: Death date
<re.Match object; span=(396, 443), match='Death date and age|1878|3|20|1814|11|25|df=y}}\n'>
y}}
%d/%m/%y
%d %B %Y
%B %d, %Y
trying: death date
None
trying: death_date
<re.Match object; span=(381, 443), match='death_date = {{Death date and age|1878|3|20|1814|>
{{Death date and age|1878|3|20|1814|11|25|df
%d/%m/%y
%d %B %Y
%B %d, %Y
this: None


In [ ]:
<re.Match object; span=(473, 503), match='Birth date|df=yes|1879|3|14}}\n'>